In [1]:
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import glob
import time
from rasterio.merge import merge
from rasterio.plot import show

In [2]:
root_dir = !pwd
root_dir = str(root_dir[0])
folder = "/merged_reprojected"
files_folder = root_dir + folder


In [3]:

# define search criteria
search_criteria = "*.tif"
search_term = os.path.join(files_folder, search_criteria)
# perform search


In [4]:
# create DF and uniwe
files = glob.glob(search_term)
dates = []
tiles = []
for i in files:
    dates.append(i[78:86])
    tiles.append(i[102:108])
df = pd.DataFrame(np.column_stack([files, dates,tiles]),columns=["files","dates","tiles"])
unique_tiles = pd.unique(df["tiles"])

In [5]:
tile_ls = []

# loop over unique dates
for tile in unique_tiles:
    print("CALCULATING: ",tile)
    # filter date
    tmp = df.loc[df["tiles"]==tile]
    tmp.reset_index(inplace=True)
    #print(tmp["files"][0])
    tile_ls.append(tmp["files"][0])

CALCULATING:  T30UXU
CALCULATING:  T30TVT
CALCULATING:  T30UVV
CALCULATING:  T30UUU
CALCULATING:  T30UVU
CALCULATING:  T30TWT
CALCULATING:  T30UWV
CALCULATING:  T30UWU
CALCULATING:  T30TXT
CALCULATING:  T30UXV


In [6]:
# open files

src_files_to_mosaic = []
for file in tile_ls:
    src = rasterio.open(file)
    src_files_to_mosaic.append(src)
    
    # perform merge
print("performing merge...")
mosaic, out_trans = merge(src_files_to_mosaic, target_aligned_pixels=True,)
    
    # save metadata
print("saving metadata...")
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                     "height": mosaic.shape[1],
                     "width": mosaic.shape[2],
                     "transform": out_trans,
                     "crs": 2154
                     }
                    )
    
# save
print("saving...")
with rasterio.open("/home/simon/CDE_UBS/thesis/data_collection/sen2/sen2_mosaic.tif", "w", **out_meta) as dest:
    dest.write(mosaic)

performing merge...
saving metadata...
saving...


In [7]:


with rasterio.open("/home/simon/CDE_UBS/thesis/data_collection/sen2/sen2_mosaic.tif", "w", **out_meta) as dest:
    dest.write(mosaic)
